# Overview of the scRNA-seq Data Processing Pipeline

This pipeline is designed to **load single-cell RNA sequencing (scRNA-seq) data**, perform preliminary data processing, save the data in Hierarchical Data Format (HDF5) as .h5ad files suitable for downstream analysis.

Furthermore, it logs essential statistics about the processing run. 
The pipeline leverages a YAML configuration file for easy adaptability and reproducibility across different datasets and environments.

# 1 - Importing libraries

In [1]:
import scanpy as sc
import os
import anndata as ad
import pandas as pd
import numpy as np
import yaml
import logging

# 2 - Declare functions to create a loading pipeline

## 2.1 - load_and_filter_scrnaseq

**Purpose:**

Loads scRNA-seq data from specified directories, filters them based on certain identifiers, and formats them into an appropriate data structure (e.g., an AnnData object for .h5ad files).

**Inputs:**

parent_directory_path (str): Path to the directory containing the scRNA-seq data files.
sample_identifiers (list): List of strings used to select specific directories or files for processing.
exclude_dirs (list): List of directory names to exclude from processing.
Outputs:

Returns a dictionary of AnnData objects or a similar data structure, indexed by sample identifier, ready for further analysis or file output.

**Mechanism:**

Iterates over subdirectories within the parent_directory_path, excluding specified directories.
For each subdirectory that includes any of the identifiers in sample_identifiers, it loads the data, typically using a library suited to reading the specific format of scRNA-seq data.
Data are potentially preprocessed or filtered based on additional criteria, and then organized into a structured format.

In [ ]:
def load_and_filter_scrnaseq(parent_directory_path, sample_identifiers, exclude_dirs=None):
    """
    Load scRNA-seq data from subdirectories containing specific identifiers and store as separate AnnData objects in dictionaries.
    
    Parameters:
    - parent_directory_path: str, path to the parent directory containing sample subdirectories
    - sample_identifiers: list, list of strings that subdirectory names must contain to be included
    - exclude_dirs: list, optional list of directory names to exclude
    
    Returns:
    - adatas: dict, dictionary of dictionaries of AnnData objects keyed by the identifiers and then sample names
    """
    if exclude_dirs is None:
        exclude_dirs = ['.ipynb_checkpoints', 'cache']
    
    # Initialize dictionary to store dictionaries of AnnData objects for each identifier
    adatas = {identifier: {} for identifier in sample_identifiers}
    
    # Create a dictionary of samples that include any of the specified identifiers
    samples = {}
    for d in os.listdir(parent_directory_path):
        dir_path = os.path.join(parent_directory_path, d)
        if os.path.isdir(dir_path) and d not in exclude_dirs:
            for identifier in sample_identifiers:
                if identifier in d:
                    samples.setdefault(identifier, []).append(dir_path)
    
    # Load data for each identifier
    for identifier, paths in samples.items():
        # Dictionary to store AnnData objects for the current identifier
        adata_dict = {}
        for path in paths:
            sample_name = os.path.basename(path)
            # Load the dataset
            adata = sc.read_10x_mtx(path, var_names='gene_symbols', cache=True)
            adata.var_names_make_unique()
            adata_dict[sample_name] = adata
        
        # Store the dictionary of AnnData objects under the corresponding identifier
        adatas[identifier] = adata_dict

    return adatas

## 2.1 - report_sample_shapes

**Purpose:**

This function generates a summary DataFrame that reports the number of cells (observations) and features (variables such as genes) for each sample in the processed scRNA-seq data. This statistical summary is crucial for initial data quality assessment and understanding the dataset's composition.

**Inputs:**

adatas (dict): A dictionary of AnnData objects, which are data structures commonly used in scRNA-seq analysis to store matrix data along with annotations of observations (cells) and variables (genes). The keys in this dictionary typically represent sample identifiers.
Outputs:

Returns a pandas DataFrame where each row corresponds to a sample. The DataFrame contains columns for:
Sample: The sample identifier.
Cells: The number of cells (observations) in the sample.
Features: The number of features (variables, typically genes) in the sample.

**Mechanism:**

The function iterates over each key-value pair in the adatas dictionary. Each key is a sample identifier, and each value is an AnnData object.
For each AnnData object, it retrieves the number of observations (n_obs) and variables (n_vars).
These metrics are then used to populate a new row in the summary DataFrame.
The populated DataFrame provides a clear and concise summary of the dataset's scale and diversity, which is essential for subsequent analytical phases.

In [ ]:
def report_sample_shapes(adatas):
    """
    Generates a report on the number of cells and features for each sample in a dictionary of dictionaries of AnnData objects.
    
    Parameters:
    - adatas: Dictionary where keys are identifiers and values are dictionaries of AnnData objects keyed by sample names.
    
    Returns:
    - DataFrame with samples and their replicates as rows, and columns for the number of cells and features.
    """
    # Initialize a list to hold data about each sample and its replicates
    sample_data = []
    
    # Iterate through each identifier in the dictionary
    for identifier, sample_dict in adatas.items():
        # Process each AnnData object in the dictionary
        for sample_name, adata in sample_dict.items():
            num_cells = adata.n_obs
            num_features = adata.n_vars
            # Append this data to the list with the sample name
            sample_data.append((f"{identifier}_{sample_name}", num_cells, num_features))
    
    # Convert the list to a DataFrame
    samples_df = pd.DataFrame(sample_data, columns=['Sample', 'Cells', 'Features'])
    
    # Set the sample name as the index of the DataFrame
    samples_df.set_index('Sample', inplace=True)
    
    return samples_df

## 2.3 - save_adatas_as_h5ad

**Purpose:**
Saves processed data in the .h5ad file format, which is commonly used in scRNA-seq analyses.

**Inputs:**

adatas (dict): Dictionary of AnnData objects, indexed by sample identifier.
directory_name (str): Path to the directory where the .h5ad files should be saved.
Outputs:

No return value. The function writes files to the disk.

**Mechanism:**

Iterates through the dictionary of AnnData objects.
For each object, constructs a filename based on its identifier and saves it to the specified directory using the .h5ad format

In [ ]:
def save_adatas_as_h5ad(adatas, directory_name='h5ad_data'):
    """
    Saves each AnnData object in a nested dictionary to .h5ad files within a specified directory.
    
    Parameters:
    - adatas: Dictionary of dictionaries containing AnnData objects. 
              Outer keys are identifiers like 'PBMC', and inner keys are sample names like 'PBMC10_matrix'.
    - directory_name: str, name of the directory where the .h5ad files will be saved.
    
    Returns:
    - None
    """
    # Create the directory if it doesn't exist
    os.makedirs(directory_name, exist_ok=True)
    
    # Iterate through each identifier and their corresponding AnnData dictionaries
    for identifier, samples in adatas.items():
        # Iterate through each sample name and its AnnData object
        for sample_name, adata in samples.items():
            # Prepare the filename by removing '_matrix' from the sample name
            filename = sample_name.replace('_matrix', '') + '.h5ad'
            # Define the full path where the .h5ad file will be saved
            file_path = os.path.join(directory_name, filename)
            # Save the AnnData object
            adata.write_h5ad(file_path)
            print(f"Saved {file_path}")  # Optional: print the path of the saved file

## 2.4 - setup_logging

**Purpose:**

Initializes the logging system for the pipeline. This function configures the logging library to write logs to a file, which helps in tracking the pipeline’s execution and debugging issues post-run.

**Inputs:**

base_directory (str): The base directory where the log file and its parent logs directory will be created.
log_filename (str): The name of the log file to write to within the logs directory.
Outputs:

No return value. The function sets up the logging environment used globally in the subsequent script.

**Mechanism:**

The function constructs a full path to the log file by appending the logs directory and the specified log filename to the base_directory.
It checks if the logs directory exists and creates it if necessary.
Logging is configured to write messages to the specified log file, with a specific format that includes the timestamp, logging level, and message. The file mode is set to overwrite (‘w’) for fresh logs each run.

In [ ]:
def setup_logging(base_directory):
    # Create the full path for the logs directory
    log_directory = os.path.join(base_directory, "logs")
    
    # Check if the logs directory exists, if not, create it
    if not os.path.exists(log_directory):
        os.makedirs(log_directory, exist_ok=True)
    
    # Define the full path for the log file within the logs directory
    log_path = os.path.join(log_directory, "pipeline.log")
    
    # Set up logging to file
    logging.basicConfig(filename=log_path, filemode='w', level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    
    logging.info("Logging initialized - if you're seeing this, logging is configured correctly")

## 2.5 - categorize_samples

**Purpose:**
This function is designed to categorize each sample based on the presence of predefined identifiers within their names. It is typically used to classify samples into groups such as "PBMC" or "CSF", which can then be used for group-specific analyses or reporting.

**Inputs:**

sample_name (str): The name of the sample which needs to be categorized.
identifiers (list): A list of strings where each string is an identifier used to categorize the samples (e.g., ["PBMC", "CSF"]).
Outputs:

Returns a string that is the category identifier found in the sample_name. If no identifiers are found, it returns 'Unknown', indicating that the sample did not match any of the predefined categories.

**Mechanism:**

The function iterates through the list of identifiers.
For each identifier, it checks if the identifier is a substring of the sample_name.
If a match is found, it returns the identifier as the category of the sample.
If no matches are found after checking all identifiers, it returns 'Unknown'.

In [ ]:
def categorize_samples(sample_name, identifiers):
    for identifier in identifiers:
        if identifier in sample_name:
            return identifier
    return 'Unknown'  # For samples that do not match any identifier

## 2.6 - log_additional_info

**Purpose:**

Enhances the logging by calculating and reporting statistical metrics for each category of samples. This includes calculating the median number of cells per category and identifying outliers based on cell counts. This function aids in quickly identifying potential data quality issues or interesting biological variations.

**Inputs:**

summary_df (DataFrame): A pandas DataFrame containing summary statistics of samples, which must include at least 'Cells' as a column and use sample names as the index.
identifiers (list): List of identifiers used to categorize samples, which are used to group the data for median and outlier calculations.
Outputs:

No return value. This function logs detailed statistics directly to the configured log file, facilitating both immediate checks and historical record-keeping.

**Mechanism:**

Category Assignment: First, the function applies the categorize_samples to assign categories to each sample based on its name (stored in the DataFrame's index).
Median Calculation: Computes the median number of cells for each category and logs this information to provide a quick overview of group statistics.
Outlier Detection:
Calculates the Interquartile Range (IQR) for cells in each category.
Defines outliers as those samples whose cell count is significantly higher or lower than the median (using a common statistical threshold of 1.5*IQR above the third quartile and below the first quartile).
Logs detailed information about any identified outliers for further examination.

In [2]:
def log_additional_info(summary_df, identifiers):
    # The index is assumed to be 'Sample', which contains the sample names
    summary_df['Category'] = summary_df.index.to_series().apply(lambda x: categorize_samples(x, identifiers))
    median_cells = summary_df.groupby('Category')['Cells'].median()
    logging.info("Median number of cells per category:\n" + median_cells.to_string())

    # Outlier detection
    for category, group in summary_df.groupby('Category'):
        median = group['Cells'].median()
        iqr = group['Cells'].quantile(0.75) - group['Cells'].quantile(0.25)
        upper_threshold = median + 1.5 * iqr
        lower_threshold = median - 1.5 * iqr
        outliers = group[(group['Cells'] > upper_threshold) | (group['Cells'] < lower_threshold)]
        if not outliers.empty:
            logging.info(f"Outliers in {category} based on cell count:\n" + outliers.to_string(index=False))

# 3 - Declare the wrapper function 'run_pipeline'

**Purpose:**
This function integrates various components of the scRNA-seq data processing pipeline, managing the flow from initial setup to the final outputs. It ensures that each step of the process is executed in sequence, and appropriate logs are generated for tracking and review.

**Inputs:**

config_path (str): The file path to the YAML configuration file which contains all necessary parameters for the pipeline's execution.

**Outputs:**

No direct return value. However, the function's execution results in several side effects including:
Processed scRNA-seq data files saved in the .h5ad format.
A comprehensive log file containing detailed information about the processing stages and data statistics.
Print statements to the console indicating the completion status or any critical messages.
Mechanism:

**Configuration Loading:**

Opens and reads the specified YAML configuration file.
Loads all relevant settings into the script, which includes directories for input data and outputs, logging configurations, and parameters for data processing functions.
- Logging Initialization:
Calls setup_logging to configure the Python logging module to write all logs to a specified file. This setup includes creating a logging directory if it does not exist.
- Directory Preparation:
Ensures that the output directory for .h5ad files exists, creating it if necessary.
Data Loading and Processing:
Executes load_and_filter_scrnaseq, which filters and loads data from specified directories into memory as structured AnnData objects. This step involves reading the data, applying predefined filters, and possibly some initial preprocessing.
- Data Summarization:
Processes the loaded data to generate a summary DataFrame that includes statistics like the number of cells and features per sample using report_sample_shapes.
- Additional Logging:
Further logs detailed statistics such as median cell counts per category and outliers using log_additional_info.
- Data Saving:
Saves the processed data into .h5ad files in the designated output directory through save_adatas_as_h5ad.
- Completion Confirmation:
Prints a message to the console to confirm that all samples have been successfully processed and saved, providing instant feedback on the operation's success.

In [3]:
def run_pipeline(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

    # Initialize logging using the parent directory path
    setup_logging(config['pipeline_settings']['parent_directory_path'])
    logging.info("Configuration loaded successfully")

    # Ensure the output directory is ready
    os.makedirs(config['pipeline_settings']['output_directory'], exist_ok=True)
    logging.info(f"Output directory {config['pipeline_settings']['output_directory']} prepared.")

    # Process data
    adatas = load_and_filter_scrnaseq(
        parent_directory_path=config['pipeline_settings']['parent_directory_path'],
        sample_identifiers=config['pipeline_settings']['sample_identifiers'],
        exclude_dirs=config['pipeline_settings']['exclude_dirs']
    )
    logging.info("Data loaded and filtered successfully")

    # Calculate and log sample information
    total_samples = 0
    for identifier, samples in adatas.items():
        num_samples = len(samples)
        total_samples += num_samples
        logging.info(f"Processed {num_samples} samples for identifier '{identifier}'")

    # Generate and log the summary DataFrame
    summary_df = report_sample_shapes(adatas)
    logging.info("Summary DataFrame of cells and features per sample:\n" + summary_df.to_string(index=True))

    # Log additional information including median and outliers
    log_additional_info(summary_df, config['pipeline_settings']['sample_identifiers'])

    # Save processed data
    save_adatas_as_h5ad(adatas, directory_name=config['pipeline_settings']['output_directory'])
    logging.info("All data saved as .h5ad files.")

    # Final user message
    print(config['output_settings']['dataframe_print_message'])

# 4 - Run pipeline

In [4]:
# Example usage
config_path = 'config.yaml'
run_pipeline(config_path)

Saved h5ad_data/PBMC10.h5ad
Saved h5ad_data/PBMC24.h5ad
Saved h5ad_data/PBMC28.h5ad
Saved h5ad_data/PBMC8.h5ad
Saved h5ad_data/PBMC32.h5ad
Saved h5ad_data/PBMC20.h5ad
Saved h5ad_data/PBMC14.h5ad
Saved h5ad_data/PBMC30.h5ad
Saved h5ad_data/PBMC27.h5ad
Saved h5ad_data/PBMC25.h5ad
Saved h5ad_data/PBMC21.h5ad
Saved h5ad_data/PBMC23.h5ad
Saved h5ad_data/PBMC31.h5ad
Saved h5ad_data/CSF1.h5ad
Saved h5ad_data/CSF31.h5ad
Saved h5ad_data/CSF23.h5ad
Saved h5ad_data/CSF21.h5ad
Saved h5ad_data/CSF3.h5ad
Saved h5ad_data/CSF29.h5ad
Saved h5ad_data/CSF25.h5ad
Saved h5ad_data/CSF13.h5ad
Saved h5ad_data/CSF7.h5ad
Saved h5ad_data/CSF27.h5ad
Saved h5ad_data/CSF30.h5ad
Saved h5ad_data/CSF14.h5ad
Saved h5ad_data/CSF20.h5ad
Saved h5ad_data/CSF32.h5ad
Saved h5ad_data/CSF16.h5ad
Saved h5ad_data/CSF12.h5ad
Saved h5ad_data/CSF28.h5ad
Saved h5ad_data/CSF24.h5ad
Saved h5ad_data/CSF10.h5ad
Saved h5ad_data/CSF26.h5ad
Saved h5ad_data/CSF8.h5ad
Saved h5ad_data/CSF4.h5ad
All samples have been successfully processed and

# 5 - Parent directory structure

The directory tree depicted below meticulously outlines the organized structure of this project.

This structure is pivotal for maintaining an efficient workflow and facilitates the initial setup, processing of scRNA-seq data, and subsequent analysis steps. Here’s a detailed breakdown of the directory tree and the functional purpose of each component:

**Initial Data Structure**

Before running the pipeline, the data should be organized according to the structure shown under the data/ directory. Each primary category (e.g., PBMC, CSF) represents a distinct dataset or experimental condition. 

PBMC/ and CSF/: These directories  house the data files necessary for processing (counts.txt, metadata.txt, matrix.mtx). This setup illustrates the typical segregation of scRNA-seq data by sample type or condition, facilitating targeted analyses and batch processing.

**Post-Pipeline Structure**

After executing the pipeline, additional directories and files are generated, structured as follows:


h5ad_data/: This directory is populated with processed data files, typically in .h5ad format, which are ready for downstream analysis or publication. The organization within this directory can be customized based on user needs or specific analysis stages.

logs/: Stores log files that record detailed execution traces, important for debugging and verifying pipeline operations. The pipeline.log file, for instance, provides insights into each step of the pipeline, including any issues encountered and statistics on the data processed.

In [8]:
def print_tree(directory, prefix=''):
    """Recursively prints a formatted tree structure of the directory path given.

    Args:
    directory (str): Path of the directory to print the structure for.
    prefix (str): Prefix to use for line formatting in recursive calls.
    """
    files = []
    directories = []
    # Separate files and directories
    for item in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, item)):
            directories.append(item)
        else:
            files.append(item)

    # Sort directories and files for consistent order
    directories.sort()
    files.sort()

    # Print directories and files
    entries = directories + files
    for i, entry in enumerate(entries):
        connector = "└──" if i == len(entries) - 1 else "├──"
        if i == len(entries) - 1:
            next_prefix = prefix + "    "
        else:
            next_prefix = prefix + "│   "

        print(f"{prefix}{connector} {entry}")
        if entry in directories:
            new_dir = os.path.join(directory, entry)
            print_tree(new_dir, prefix=next_prefix)

# Example usage:
print_tree('/Users/felipe/Desktop/Portfolio_FNV/PD_scRNAseq_PBMC/') 

├── .ipynb_checkpoints
│   ├── Untitled-checkpoint.ipynb
│   ├── config-checkpoint.yaml
│   └── scRNA-seq_PD_PBMC-checkpoint.ipynb
├── CSF10_matrix
│   ├── .ipynb_checkpoints
│   ├── .barcodes.tsv.icloud
│   ├── .matrix.mtx.icloud
│   └── genes.tsv
├── CSF12_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF13_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF14_matrix
│   ├── .ipynb_checkpoints
│   ├── .barcodes.tsv.icloud
│   ├── .matrix.mtx.icloud
│   └── genes.tsv
├── CSF16_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF1_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF20_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF21_matrix
│   ├── .ipynb_checkpoints
│   ├── .barcodes.tsv.icloud
│   ├── .matrix.mtx.icloud
│   └── genes.tsv
├── CSF23_matrix
│   ├── .matrix.mtx.icloud
│   ├── barcodes.tsv
│   └── genes.tsv
├── CSF24_matrix
│

# 6 - Improvement suggestions for next versions of this pipeline

**Handling No Specific Substrings**

To handle situations where **no specific substrings** are provided (i.e., no cell type names in the folder names):

Configuration Setup: Define a default behavior when sample_identifiers is empty or a specific keyword like 'ALL' is used.